In [31]:
import numpy as np
import pandas as pd
import datetime
import math

In [34]:
price = pd.read_csv('dam_out.csv')
price['date']=pd.to_datetime(price['date'],format="%Y-%m-%d")#"%d/%m/%Y")
price=price.drop(['date.1'],axis=1)
price=price.set_index([pd.DatetimeIndex(price['date']),'hour'])
price

date  hourlyid  market   tradedate   price  volume  \
date       hour                                                            
2016-01-01 0    2016-01-01     26345       1  2016-01-01   40.10   477.5   
           1    2016-01-01     26346       1  2016-01-01   40.08   486.6   
           2    2016-01-01     26347       1  2016-01-01   40.06   495.6   
           3    2016-01-01     26348       1  2016-01-01   40.08   496.6   
           4    2016-01-01     26349       1  2016-01-01   40.07   502.9   
...                    ...       ...     ...         ...     ...     ...   
2021-04-29 4    2021-04-29     73029       1  2021-04-29   39.97   126.1   
           12   2021-04-29     73037       1  2021-04-29   68.06    71.0   
2021-04-30 0    2021-04-30     73049       1  2021-04-30   44.04    59.6   
           8    2021-04-30     73057       1  2021-04-30  190.00    72.0   
           17   2021-04-30     73066       1  2021-04-30   67.97   114.2   

                 year  month  day  day_of_week  month_no    hour_ref  
date       hour                                                       
2016-01-01 0     2016      1    1            4   2016.01   weekday_0  
           1     2016      1    1            4   2016.01   weekday_1  
           2     2016      1    1            4   2016.01   weekday_2  
           3     2016      1    1            4   2016.01   weekday_3  
           4     2016      1    1            4   2016.01   weekday_4  
...               ...    ...  ...          ...       ...         ...  
2021-04-29 4     2021      4   29            3   2021.04   weekday_4  
           12    2021      4   29            3   2021.04  weekday_12  
2021-04-30 0     2021      4   30            4   2021.04   weekday_0  
           8     2021      4   30            4   2021.04   weekday_8  
           17    2021      4   30            4   2021.04  weekday_17  

[46728 rows x 12 columns]

In [35]:
flow = pd.read_csv('flow.csv')

flow['date']=pd.to_datetime(flow['flowdate'],format="%Y-%m-%d")#"%d/%m/%Y")
flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek
flow['month_no']=flow['year']+flow['month']/100

flow['hour_ref'] = flow.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=price['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.reset_index()
flow

,date,hour,flow,from,to,year,month,day,day_of_week,month_no,hour_ref,price,value
0,2016-01-02,13,50,BOT,ZIMA,2016,1,2,5,2016.01,weekend_13,50.05,2502.50
1,2016-01-02,14,63,ZIMA,ZIM,2016,1,2,5,2016.01,weekend_14,50.31,3169.53
2,2016-01-02,15,263,ZIM,ZAMZ,2016,1,2,5,2016.01,weekend_15,50.49,13278.87
3,2016-01-02,16,100,MOZN_EDM,ZIMA,2016,1,2,5,2016.01,weekend_16,56.07,5607.00
4,2016-01-02,17,120,ZIM,ZAMZ,2016,1,2,5,2016.01,weekend_17,56.06,6727.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
333320,2021-03-31,19,3,RSAN,RSAS,2021,3,31,2,2021.03,weekday_19,148.01,444.03
333321,2021-03-31,20,13,RSAN,BOT,2021,3,31,2,2021.03,weekday_20,139.06,1807.78
333322,2021-03-31,21,33,MOZS,RSAN,2021,3,31,2,2021.03,weekday_21,89.03,2937.99
333323,2021-03-31,22,43,ZIMA,ZIM,2021,3,31,2,2021.03,weekday_22,36.98,1590.14


In [36]:
f_to=flow[['date','hour','flow','value','to','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','from','year','month','day','day_of_week','month_no','hour_ref']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,node,year,month,day,day_of_week,month_no,hour_ref
0,2016-01-02,13,50,2502.50,ZIMA,2016,1,2,5,2016.01,weekend_13
1,2016-01-02,14,63,3169.53,ZIM,2016,1,2,5,2016.01,weekend_14
2,2016-01-02,15,263,13278.87,ZAMZ,2016,1,2,5,2016.01,weekend_15
3,2016-01-02,16,100,5607.00,ZIMA,2016,1,2,5,2016.01,weekend_16
4,2016-01-02,17,120,6727.20,ZAMZ,2016,1,2,5,2016.01,weekend_17
...,...,...,...,...,...,...,...,...,...,...,...
333320,2021-03-31,19,-3,-444.03,RSAN,2021,3,31,2,2021.03,weekday_19
333321,2021-03-31,20,-13,-1807.78,RSAN,2021,3,31,2,2021.03,weekday_20
333322,2021-03-31,21,-33,-2937.99,MOZS,2021,3,31,2,2021.03,weekday_21
333323,2021-03-31,22,-43,-1590.14,ZIMA,2021,3,31,2,2021.03,weekday_22


In [37]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='MOZ'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [38]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week','month_no','hour_ref']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows

,date,hour,node,year,month,day,day_of_week,month_no,hour_ref,index,flow,value
0,2016-01-01,0,ZAMC,2016,1,1,4,2016.01,weekday_0,5607,160,6416.00
1,2016-01-01,0,ZAMZ,2016,1,1,4,2016.01,weekday_0,11320,10,401.00
2,2016-01-01,0,ZIM,2016,1,1,4,2016.01,weekday_0,5713,-170,-6817.00
3,2016-01-01,1,ZAMC,2016,1,1,4,2016.01,weekday_1,5714,160,6412.80
4,2016-01-01,1,ZAMZ,2016,1,1,4,2016.01,weekday_1,11322,10,400.80
...,...,...,...,...,...,...,...,...,...,...,...,...
289677,2021-03-31,22,ZAMC,2021,3,31,2,2021.03,weekday_22,333292,43,1590.14
289680,2021-03-31,23,BOT,2021,3,31,2,2021.03,weekday_23,666156,81,2993.76
289681,2021-03-31,23,MOZ,2021,3,31,2,2021.03,weekday_23,666292,-164,-6061.44
289682,2021-03-31,23,NAM,2021,3,31,2,2021.03,weekday_23,333309,42,1552.32


In [39]:
# flows.loc[(flows['date']=='2016-01-15') & (flows['hour']==0)]

In [40]:
flow_monthly=flows.groupby(['month_no','year','month','hour', 'hour_ref','node']).sum().reset_index(['hour','month','year'])
flow_monthly.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly=flow_monthly.drop(['day_of_week','day'],axis=1)
flow_monthly

year  month  hour    index  flow_sum     value
month_no hour_ref   node                                                
2016.01  weekday_0  BOT   2016      1     0    16865       -71  -3460.97
                    MOZ   2016      1     0   143656     -1319 -65858.37
                    NAM   2016      1     0     4724        50   2409.80
                    RSA   2016      1     0     7797       -76  -3734.68
                    ZAMC  2016      1     0    49637      1391  65631.17
...                        ...    ...   ...      ...       ...       ...
2021.03  weekend_23 BOT   2021      3    23  1328484        63   2426.80
                    ESW   2021      3    23   995479        98   3434.32
                    MOZ   2021      3    23  7942668     -1199 -50797.77
                    NAM   2021      3    23  2648436       503  21847.38
                    ZAMC  2021      3    23  2647222       535  23089.27

[18658 rows x 6 columns]

In [41]:
fm=flow_monthly.reset_index()

fm.groupby(['month_no','hour_ref']).sum().sort_values('flow_sum')

fm.loc[(fm['month_no']==2017.01) & (fm['hour_ref']=='weekend_20')]['flow_sum'].sum()

0

In [42]:
flow_monthly.to_csv('dam_flow_monthly.csv')

In [ ]:
flowa=flow.copy().rename(columns={'to':'from','from':'to'})
flow.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    ) 
    ,['to','from']]=flowa.loc[
    (
        (flow['from']=='RSAN') & (flow['to']=='MOZS') |
        (flow['from']=='MOZS') & (flow['to']=='RSAN') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='ZIMA') |
        (flow['from']=='ZIMA') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_HCB') & (flow['to']=='MOZN_EDM') |
        (flow['from']=='MOZN_EDM') & (flow['to']=='MOZN_HCB')
    )     
    ,['to','from']]